In [141]:
import numpy
import random
from collections import defaultdict
import urllib
import math
import random
import collections
import string
import csv
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import sklearn.metrics
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords

nltk.download('stopwords')

NameError: name 'nltk' is not defined

In [2]:
def parseLabeledData(path):
    file=open(path, 'r')
    dataList = []
    for line in csv.reader(file):
        if len(line) == 4:
            dataList.append(
                {"asin":line[0], 
                 "question":line[1],
                 "review":line[2],
                 "label":line[3]}
            )     
    return dataList
        

print("Reading labeled data...")
data = parseLabeledData("/Users/Silvia/Desktop/LabelledData.csv")
queries = [d['question'] for d in data]
reviews = [d['review'] for d in data]
labels = [d['label'] for d in data]
print("done")

Reading labeled data...
done


In [3]:
def parseAllQueries(path):
    file = open(path, 'r')
    dataList = defaultdict(lambda: [])
    for line in file:
        line = eval(line)
        dataList[line["asin"]].append(line["question"])
      
    return dataList

def parseAllReviews(path):
    file = open(path, 'r')
    dataList = defaultdict(lambda: [])
    for line in file:
        line = eval(line)
        dataList[line["asin"]].append(line["reviewText"])
      
    return dataList

print("Reading all reviews & all questions...")
allReviews = parseAllReviews("/Users/Silvia/Downloads/reviews.json")
allQuestions = parseAllQueries("/Users/Silvia/Downloads/qa.json")

# do we have to remove questions that have no reviews or reviews that have no questions??
docSet = [review for review in allReviews] + [question for question in allQuestions]

print("done")

Reading all reviews & all questions...
done


In [4]:
def findCommonWords():
    allWords = defaultdict(int)
    for r in allReviews.values():
        for review in r:
            exclude = set(string.punctuation)
            review = ''.join(ch for ch in review if ch not in exclude)
            for w in review.lower().split():
                allWords[w] += 1

    for q in allQuestions.values():
        for question in q:
            exclude = set(string.punctuation)
            question = ''.join(ch for ch in question if ch not in exclude)
            for w in question.lower().split():
                allWords[w] += 1
    
    
    return sorted(allWords, key=lambda x: -allWords[x])[:1000]

In [5]:
commonWords = findCommonWords()

In [6]:
# @param a word whose frequency in the document we are calculating
# @param document a string of a review or a question
# @return the frequency of term in document div length of document

def tf(term, document):
    count = collections.defaultdict(int)
    for word in document.split():
        count[word] += 1

    return count[term]/len(document.split())

In [7]:
def idf(term):
    count = 0         
    for doc in docSet:
        if term in doc.lower():
            count += 1
    return math.log(len(docSet)/(count+1))

In [8]:
def wordToIndex(term):
    if term in commonWords:
        return commonWords.index(term)
    else:
        return -1

In [142]:
def numCommonWords(review, question):
    print(stopwords.words('english'))
    
    filtered_words = [word for word in review.split() if word not in stopwords.words('english')]
    num = 0
    words = set(filtered_words)
    for word in words:
        if word in question:
            num = num + 1
    #print(num)
    return num

numCommonWords("a s d f g xx a","a a xx")

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  Searched in:
    - '/Users/Silvia/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.6/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.6/lib/nltk_data'
**********************************************************************


In [127]:
def feature(review, question):
    feat = [1]
    
    feat.append(numCommonWords(review, question))
    
    return feat

In [131]:
def normalize(featList):
    
    max = 0
    min = float('inf')
    for feat in featList:
        if feat > max: max = feat
        if feat < min: min = feat        
    
    for i in range(0,len(featList)-1):
        if (max - min) == 0: 
            max = 1
            min = 0
        featList[i] = (featList[i] - min) / (max - min)

    return featList
    
    
    

In [132]:
def constrain(elems):
    for elem in elems:
        if elem > 0.5: yield 1
        else: yield 0

In [135]:
def linearRegression():
    X = [feature(d["review"], d["question"]) for d in data]
    
    for j in range(0,len(X[0])):
        featList = []
        for i in range(0,len(X)-1):
            featList.append(X[i][j])
        featList = normalize(featList)
        for i in range(0,len(X)-1):
            X[i][j] = featList[i]
            
    y = [1 if l == "y" else 0 for l in labels]
    
    keys = list(range(1, len(labels)))
    points = dict(zip(keys, zip(X, y)))
    random.shuffle(keys)
    X = [points[key][0] for key in keys]
    y = [points[key][1] for key in keys]
    
    X_train = X[:len(X)//2]
    y_train = y[:len(y)//2]
    
    X_test = X[len(X)//2:]
    y_test = y[len(y)//2:]
    
    #theta,residuals,rank,s = numpy.linalg.lstsq(X_train, y_train, rcond=None)
    
    #X_train = numpy.matrix(X_train)
    #y_train = numpy.matrix(y_train).T
    #theta = numpy.matrix(theta).T
    #y_hatTrain = X_train * theta
    
    #X_test = numpy.matrix(X_test)
    #y_test = numpy.matrix(y_test).T
    #y_hatTest = X_test * theta
    
    # Find mean of Training and Test
    #y_barTrain = (sum(y_train) / len(y_train))
    #y_barTest = (sum(y_test) / len(y_test))
    
    #print(r2_score(y_test, y_hatTest))
    
    #clf = LinearSVC(random_state=0, C=1.0)
    #clf.fit(X_train, y_train)
    #y_hatTrain = clf.predict(X_train)
    #y_hatTest = clf.predict(X_test)
    
    # the accuracy score is very .77. Regardless of the C we chose. But the C score DID, in fact, impact our
    # training accuracy
    
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_hatTrain = lr.predict(X_train)
    y_hatTest = lr.predict(X_test)
    
    #print(X_train)
    
    y_hatTrain = list(constrain(y_hatTrain))
    y_hatTest = list(constrain(y_hatTest))
    
    
    print(y_hatTest)
    
    print("train:")
    print("accuracy: ", sklearn.metrics.accuracy_score(y_train, y_hatTrain), "\t", \
          "precision: ", sklearn.metrics.precision_score(y_train, y_hatTrain), "\t", \
          "recall: ", sklearn.metrics.recall_score(y_train, y_hatTrain))
          
    print("test:")
    print("accuracy: ", sklearn.metrics.accuracy_score(y_test, y_hatTest), "\t", \
          "precision: ", sklearn.metrics.precision_score(y_test, y_hatTest), "\t", \
          "recall: ", sklearn.metrics.recall_score(y_test, y_hatTest))
    

    
    
linearRegression()
    
    


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
train:
accuracy:  0.8050847457627118 	 precision:  0.0 	 recall:  0.0
test:
accuracy:  0.8305084745762712 	 precision:  1.0 	 recall:  0.047619047619047616


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
